In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Levenshtein as lev
sns.set()

In [105]:
#Levantamos los datasets en crudo (archivos .csv)
df_clientes     =pd.read_csv('../Datasets/Clientes.csv'   , encoding='latin-1', sep=';')
df_compras      =pd.read_csv('../Datasets/Compra.csv'     , encoding='latin-1', sep=',')
df_gastos       =pd.read_csv('../Datasets/Gasto.csv'      , encoding='latin-1', sep=',')
df_localidades  =pd.read_csv('../Datasets/Localidades.csv', encoding='latin-1', sep=',')
df_proveedores  =pd.read_csv('../Datasets/Proveedores.csv', encoding='latin-1', sep=',')
df_sucursales   =pd.read_csv('../Datasets/Sucursales.csv' , encoding='latin-1', sep=';')
df_ventas       =pd.read_csv('../Datasets/Venta.csv'      , encoding='latin-1', sep=',')
df_canalventa   =pd.read_excel('../Datasets/CanalDeVenta.xlsx')
df_tipogasto    =pd.read_csv('../Datasets/TiposDeGasto.csv', encoding='latin-1', sep=',')

Provincias Normalizadas

In [106]:
df_provincias = pd.read_csv('../Datasets/provincias.csv', sep=',', encoding='utf-8')
provincias_normalizadas = list(df_provincias['nombre_completo'].unique())

#### Renombrando columnas

In [107]:
#Clientes
cols_nuevas = ['idCliente','provincia', 'nom_y_ape', 'domicilio', 'tel', 'edad', 'localidad', 'lat', 'long', 'col10']
df_clientes.columns = cols_nuevas

#Compras
cols_nuevas = ['idCompra','fecha', 'año', 'mes', 'periodo', 'idProducto','cantidad', 'precio', 'idProveedor']
df_compras.columns = cols_nuevas

#Gastos
cols_nuevas = ['idGasto', 'idSucursal', 'idTipoGasto', 'fecha', 'monto']
df_gastos.columns = cols_nuevas

#Localidades
cols_nuevas = ['categoria','lat','lon','idDepartamento','nombreDepartamento','fuente','idLocalidad','idLocalidadCensal','nombreLocalidadCensal','idMunicipio','nombreMunicipio','nombreLocalidad','idProvincia','nombreProvincia']
df_localidades.columns = cols_nuevas

#Proveedores
cols_nuevas = ['idProveedor','nombre', 'direccion', 'ciudad', 'estado', 'pais', 'departamento']
df_proveedores.columns = cols_nuevas

#Sucursales
cols_nuevas = ['idSucursal','sucursal', 'direccion', 'localidad', 'provincia', 'lat', 'long']
df_sucursales.columns = cols_nuevas

#Ventas
cols_nuevas = ['idVenta','fechaVenta','fechaEntrega','idCanal','idCliente','idSucursal','idEmpleado','idProducto','precio','cantidad']
df_ventas.columns = cols_nuevas

#Tipo de Gasto
cols_nuevas = ['idTipoGasto', 'descripcion', 'montoAprox']
df_tipogasto.columns = cols_nuevas

#Canal de Venta
cols_nuevas = ['codigo', 'descripcion']
df_canalventa.columns = cols_nuevas

## Clientes

Eliminamos la columna 'col10' ya que no aporta nada al dataset.

In [108]:
df_clientes.drop(columns='col10', inplace=True)

Capitalizamos la columna 'nom_y_ape', 'provincia', 'localidad' y 'domicilio'.

In [109]:
columnas = ['provincia', 'nom_y_ape', 'localidad', 'domicilio']
for col in columnas:
    df_clientes[col] = df_clientes[col].str.title()

Llenamos los nulos con el promedio para poder realizar la conversion.<br>
Convertimos a entero el campo 'edad'.

In [110]:
df_clientes['edad'].fillna(int(df_clientes['edad'].mean()), inplace=True)

In [111]:
df_clientes['edad'] = df_clientes['edad'].astype('int')

Latitud y Longitud

In [112]:
df_clientes['lat'] = df_clientes['lat'].str.replace(',', '')
df_clientes['long'] = df_clientes['long'].str.replace(',', '')

In [113]:
df_clientes['lat'] = pd.to_numeric(df_clientes['lat'], errors='coerce', downcast='float')

Normalizamos las provincias

In [121]:
m = df_clientes['provincia'].isna()

In [124]:
for prov_normal in provincias_normalizadas:
    for i, prov in enumerate(df_clientes.loc[(m==False),'provincia']):
        porc_parecido = lev.ratio(prov_normal.lower(), prov.lower())
        if (porc_parecido > 0.50):
            df_clientes['provincia'][i] = prov_normal

C:\Users\Laucha\AppData\Local\Temp\ipykernel_3792\2905108201.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clientes['provincia'][i] = prov_normal


## Compras

Columna 'precio' convertida a float.<br>

In [ ]:
m = df_compras['precio'].isna()
df_compras.loc[(m==False), 'precio'] = df_compras['precio'].astype('float')

Llenamos los nulos con el promedio

In [ ]:
df_compras['precio'].fillna(df_compras['precio'].mean(), inplace=True)

Columnas 'idCompra', 'año', 'mes', 'periodo', 'idProducto', 'cantidad', 'idProveedor' a entero.

In [ ]:
cols = ['idCompra', 'año', 'mes', 'periodo', 'idProducto', 'cantidad', 'idProveedor']
for col in cols:
    m = ((df_compras[col].isna()) | (df_compras[col]==''))
    df_compras.loc[ (m==False), col] = df_compras[col].astype('int')

Seteamos en cero (0) la columna 'cantidad' donde el precio es Nulo.

In [ ]:
df_compras.loc[m, 'cantidad'] = 0

## Ventas

In [ ]:
df_ventas

,idVenta,fechaVenta,fechaEntrega,idCanal,idCliente,idSucursal,idEmpleado,idProducto,precio,cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2.0
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3.0
...,...,...,...,...,...,...,...,...,...,...
46175,47596,2020-11-30,2020-12-09,1,193,26,3032,42850,279.00,2.0
46176,47597,2020-11-30,2020-12-07,3,333,26,3193,42799,1536.04,2.0
46177,47598,2020-11-30,2020-12-07,1,2389,27,3667,42916,1356.00,2.0
46178,47599,2020-11-30,2020-12-04,1,1067,29,3836,42935,2456.00,1.0


## Gastos

## Localidades

Normalizar Provincias

In [ ]:
for prov_normal in provincias_normalizadas:
    for i, prov in enumerate(df_localidades['nombreProvincia']):
        porc_parecido = lev.ratio(prov_normal, prov)
        if porc_parecido > 0.50:
            df_localidades['nombreProvincia'][i] = prov_normal

In [ ]:
df_localidades

,categoria,lat,lon,idDepartamento,nombreDepartamento,fuente,idLocalidad,idLocalidadCensal,nombreLocalidadCensal,idMunicipio,nombreMunicipio,nombreLocalidad,idProvincia,nombreProvincia
0,Localidad simple,-35.033073,-60.280620,6021.0,Alberti,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6,Provincia de Buenos Aires
1,Localidad simple,-34.868119,-60.393971,6021.0,Alberti,INDEC,6021020000,6021020,Coronel SeguÃ­,60021.0,Alberti,CORONEL SEGUI,6,Provincia de Buenos Aires
2,Componente de localidad compuesta,-35.068014,-60.402597,6021.0,Alberti,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6,Provincia de Buenos Aires
3,Localidad simple,-35.124382,-60.220061,6021.0,Alberti,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6,Provincia de Buenos Aires
4,Localidad simple,-35.109618,-60.070551,6021.0,Alberti,INDEC,6021050000,6021050,Villa GrisolÃ­a,60021.0,Alberti,VILLA GRISOLIA,6,Provincia de Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4137,Localidad simple,-34.688216,-58.957430,6364.0,General RodrÃ­guez,INDEC,6364030005,6364030,General RodrÃ­guez,60364.0,General RodrÃ­guez,BARRIO RUTA 24 KILOMETRO 10,6,Provincia de Buenos Aires
4138,Localidad simple,-53.787047,-67.713235,94008.0,RÃ­o Grande,INDEC,94008010000,94008010,RÃ­o Grande,940007.0,RÃ­o Grande,RIO GRANDE,94,"Provincia de Tierra del Fuego, Antártida e Isl..."
4139,Localidad simple,-54.511387,-67.195804,94008.0,RÃ­o Grande,INDEC,94008020000,94008020,Tolhuin,942007.0,Tolhuin,TOLHUIN,94,"Provincia de Tierra del Fuego, Antártida e Isl..."
4140,Localidad simple,-54.637686,-67.766941,94015.0,Ushuaia,INDEC,94015010000,94015010,Laguna Escondida,NaN,NaN,LAGUNA ESCONDIDA,94,"Provincia de Tierra del Fuego, Antártida e Isl..."
